<img height="60px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="20px" vspace="5px">

# Softmax-Classifier for CIFAR-10

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/learning-stack/Colab-ML-Playbook/blob/master/How%20to%20Build%20a%20Simple%20Image%20Recognition%20System%20with%20TensorFlow/How%20to%20Build%20a%20Simple%20Image%20Recognition%20System%20with%20TensorFlow%20-%20Part%201.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/learning-stack/Colab-ML-Playbook/blob/master/How%20to%20Build%20a%20Simple%20Image%20Recognition%20System%20with%20TensorFlow/How%20to%20Build%20a%20Simple%20Image%20Recognition%20System%20with%20TensorFlow%20-%20Part%201.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import time

In [15]:
# Import custom library from my github

import urllib.request

def download(url):
    filename = url.split('/')[-1]
    print ('Downloading', filename)
    f =  urllib.request.urlopen(url)
    data = f.read()
    f.close()
    with open(filename, 'wb') as myfile:
        myfile.write(data)

# get .py file from repository
download('https://raw.githubusercontent.com/learning-stack/Colab-ML-Playbook/master/How%20to%20Build%20a%20Simple%20Image%20Recognition%20System%20with%20TensorFlow/data_helpers.py')

# verify the file if necessary
# print(open('data_helpers.py').read())

import data_helpers


## Parameter definitions

In [0]:
batch_size = 1000
learning_rate = 0.005
max_steps = 1000


beginTime = time.time()

# Uncommenting this line removes randomness
# You'll get exactly the same result on each run
# np.random.seed(1)

## Prepare data

In [17]:
data_sets = data_helpers.load_data()

FileNotFoundError: ignored

## Prepare the TensorFlow graph

In [0]:
# Define input placeholders
images_placeholder = tf.placeholder(tf.float32, shape=[None, 3072])
labels_placeholder = tf.placeholder(tf.int64, shape=[None])

In [0]:
# Define variables (these are the values we want to optimize)
weights = tf.Variable(tf.zeros([3072, 10]))
biases = tf.Variable(tf.zeros([10]))

In [0]:
# Define the classifier's result
logits = tf.matmul(images_placeholder, weights) + biases

In [0]:
# Define the loss function
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
  labels=labels_placeholder))

In [0]:
# Define the training operation
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [0]:
# Operation comparing prediction with true label
correct_prediction = tf.equal(tf.argmax(logits, 1), labels_placeholder)

In [0]:
# Operation calculating the accuracy of our predictions
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Run the TensorFlow graph

In [0]:
with tf.Session() as sess:
  # Initialize variables
  sess.run(tf.global_variables_initializer())
    
  # Repeat max_steps times
  for i in range(max_steps):

    # Generate input data batch
    indices = np.random.choice(data_sets['images_train'].shape[0], batch_size)
    images_batch = data_sets['images_train'][indices]
    labels_batch = data_sets['labels_train'][indices]

    # Periodically print out the model's current accuracy
    if i % 100 == 0:
      train_accuracy = sess.run(accuracy, feed_dict={
        images_placeholder: images_batch, labels_placeholder: labels_batch})
      print('Step {:5d}: training accuracy {:g}'.format(i, train_accuracy))

    # Perform a single training step
    sess.run(train_step, feed_dict={images_placeholder: images_batch,
      labels_placeholder: labels_batch})

  # After finishing the training, evaluate on the test set
  test_accuracy = sess.run(accuracy, feed_dict={
    images_placeholder: data_sets['images_test'],
    labels_placeholder: data_sets['labels_test']})
  print('Test accuracy {:g}'.format(test_accuracy))

endTime = time.time()

Step     0: training accuracy 0.108
Step   100: training accuracy 0.282
Step   200: training accuracy 0.229
Step   300: training accuracy 0.281
Step   400: training accuracy 0.293
Step   500: training accuracy 0.297
Step   600: training accuracy 0.318
Step   700: training accuracy 0.313
Step   800: training accuracy 0.272
Step   900: training accuracy 0.281
Test accuracy 0.2715


In [0]:
print('Total time: {:5.2f}s'.format(endTime - beginTime))

Total time: 44.58s
